In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, IntegerType, FloatType, StringType

spark = SparkSession \
        .builder \
        .appName("frequent_itemsets") \
        .getOrCreate()

24/11/28 17:31:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [21]:
data = spark.read.parquet("data_with_topics.parquet").dropna()

In [22]:
data.show(5)

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+---------+-------+-----+
|     id|        cleaned_text|              topic0|              topic1|              topic2|              topic3|              topic4|              topic5|              topic6|              topic7|              topic8|emo_pos|emo_neg|emo_anx|emo_anger|emo_sad|moral|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+-------+-------+---------+-------+-----+
|1001mlo|hi im a 24 f who ...| 0.11968858380250595|7.442736887136972E-4| 0.06107536248365846|9.333486297156222E-4|0.052468338961077526|9.792222275302406E-4|0.001049030320834...| 0.0632594240943850

In [30]:
mapping_topics = {
    'topic0': 0,
    'topic1': 1,
    'topic2': 2,
    'topic3': 3,
    'topic4': 4,
    'topic5': 5,
    'topic6': 6,
    'topic7': 7,
    'topic8': 8}

mapping_emotions = {
    'emo_pos': 11,
    'emo_anx': 12,
    'emo_anger': 13,
    'emo_sad': 14,
    'moral': 15}

def create_items(input_df, mapping, threshold, output_col):
    # Create a new column 'positive_columns' with the integers where values are greater than 1
        df = input_df.withColumn(
        "relevant",
        F.array([
        F.when(F.col(col) > threshold, F.lit(value)).otherwise(None) 
        for col, value in mapping.items()]))

        # Filter out None values from the array
        df = df.withColumn(f"{output_col}", F.expr("filter(relevant, x -> x is not null)"))
        
        return df 

In [39]:
df_topics = create_items(data, mapping_topics, 0.2, "relevant_topics")
df_liwc = create_items(df_topics, mapping_emotions, 0.1, "relevant_liwc_scores")
df_topics.select('relevant_topics').show(5)
df_liwc.select('relevant_topics', 'relevant_liwc_scores').show(5)

df = df_liwc.withColumn("relevant_attr", F.concat(df_liwc["relevant_topics"], df_liwc["relevant_liwc_scores"]))
df.select('relevant_attr').show(5)

+---------------+
|relevant_topics|
+---------------+
|            [8]|
|         [0, 4]|
|         [1, 7]|
|         [3, 7]|
|         [6, 8]|
+---------------+
only showing top 5 rows

+---------------+--------------------+
|relevant_topics|relevant_liwc_scores|
+---------------+--------------------+
|            [8]|    [11, 12, 13, 15]|
|         [0, 4]|        [11, 13, 15]|
|         [1, 7]|        [11, 13, 15]|
|         [3, 7]|                [11]|
|         [6, 8]|                  []|
+---------------+--------------------+
only showing top 5 rows

+-------------------+
|      relevant_attr|
+-------------------+
|[8, 11, 12, 13, 15]|
| [0, 4, 11, 13, 15]|
| [1, 7, 11, 13, 15]|
|         [3, 7, 11]|
|             [6, 8]|
+-------------------+
only showing top 5 rows



In [41]:
from pyspark.ml.fpm import FPGrowth

fp = FPGrowth(minConfidence=0.5, minSupport=0.01)
fpm = fp.fit(df.select(df.relevant.alias('items')))
fpm.associationRules.orderBy("lift", "confidence", ascending=False).show(truncate=False)

24/11/28 18:03:07 ERROR Executor: Exception in task 6.0 in stage 141.0 (TID 1367)
org.apache.spark.SparkException: Items in a transaction must be unique but got WrappedArray(11, null, 13, null, 15).
	at org.apache.spark.mllib.fpm.FPGrowth.$anonfun$genFreqItems$1(FPGrowth.scala:249)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:197)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.run(Task.scala:136)
	at org.apache.spark.executor.Executor$

Py4JJavaError: An error occurred while calling o2151.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 141.0 failed 1 times, most recent failure: Lost task 6.0 in stage 141.0 (TID 1367) (midway3-0264.rcc.local executor driver): org.apache.spark.SparkException: Items in a transaction must be unique but got WrappedArray(11, null, 13, null, 15).
	at org.apache.spark.mllib.fpm.FPGrowth.$anonfun$genFreqItems$1(FPGrowth.scala:249)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:197)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:63)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTas